In [3]:
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
from Bio import SeqIO
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
import pickle


np.set_printoptions(suppress=True)

In [ ]:
class Data():
    def __init__(self):
        self.seq_length = 101
        self.data_path = "with_reverse/RBBP5"
        self.negative_list = []
        self.positive_list = []
        self.positivecount = 0
        self.negativecount = 0
        self.onehot_encoded_positive = []
        self.onehot_encoded_negative = []
        
    def onehot_encode_sequences(self, sequences):
        onehot = []
        mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3, 'U': 3, 'N':4}
        for sequence in sequences:
            arr = np.zeros((len(sequence), 5)).astype("float")
            for (i, letter) in enumerate(sequence):
                arr[i, mapping[letter]] = 1.0
            onehot.append(arr)
        return onehot

    def reverse_complement(self, dna):
        complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A','N': 'N'}
        return ''.join([complement[base] for base in dna[::-1]])
        
        
        
    def fasta_prepare_data(self):
        path = os.path.join(self.data_path, "RBBP5_positive_training.fa")
        f = open(path, 'r')
        for line in f:
            self.positive_list.append(line)
            self.positivecount += 1
        #print(self.positivecount)
        self.positive_list = list(map(lambda x:x.strip(), self.positive_list))
        onehot_positive = self.onehot_encode_sequences(self.positive_list)


        path = os.path.join(self.data_path, "RBBP5_negative_training.fa")
        f = open(path, 'r')
        for line in f:
            self.negative_list.append(line)
            self.negativecount += 1
        #print(self.negativecount)
        self.negative_list = list(map(lambda x:x.strip(), self.negative_list))
        onehot_negative = self.onehot_encode_sequences(self.negative_list)
        
        combinedlist = []
        
        for sequence in onehot_positive:
            combinedlist.append([sequence, [0,1]])
        for sequence in onehot_negative:
            combinedlist.append([sequence,[1,0]])
        
        np.random.shuffle(combinedlist)
        
        
        for data in combinedlist:
            data[0] = np.transpose(data[0])
            
        
            
        return combinedlist
    
    

    def seq_prepare_data(self):
        #----------------------------------------------------------------------------------------------
        # positive data
        path = "/usr/local/lib/EzGeno/Backbone_model_data/"
        file_counter = 0
        
        for file in os.listdir(path):
            new_path = os.path.join(path, file)


            if new_path.lower().endswith('seq'):
                file_counter += 1
                #print("Number of files processed: %s" %(file_counter))
                # print(new_path)
                with open(new_path) as outputfile:
                    for sequence in outputfile:

                        if sequence[0] != 'A':
                            pass
                        else:
                            self.positive_list.append(sequence[17:118])
                            self.positive_list.append(self.reverse_complement(sequence[17:118]))

        print("shuffling")                    
        random.shuffle(self.positive_list)
        print("Number of positive sequences: %s" %(len(self.positive_list)))


        #-----------------------------------------------------------------------------------------------
        #negative data

        negative_path = "/usr/local/lib/EzGeno/negative_fasta.fa"
        test_sequences = SeqIO.parse(open('negative_fasta.fa'),'fasta')
        with open('negative_fasta.fa') as out_file:
            for sequence in test_sequences:
                self.negative_list.append(str(sequence.seq).upper())

        print("Number of negative sequences: %s" %(len(self.negative_list)))
        
        
        return self.positive_list, self.negative_list

        
        
my_data = Data()
data1 = my_data.fasta_prepare_data()
#positive_list, negative_list = my_data.seq_prepare_data()






In [ ]:
#---------------parse to 10000 sequences per file----------------------

file_counter = 1
for i in range(0, len(positive_list), 10000):
    pos_filename = "processed/positive/positive_%d.fa" % file_counter
    neg_filename = "processed/negative/negative_%d.fa" % file_counter
    with open(pos_filename, 'w')as f:
        for item in positive_list[i:i + 10000]:
            f.write("%s\n" % item)
    with open(neg_filename, 'w')as f:
        for item in negative_list[i:i + 10000]:
            f.write("%s\n" % item)  
    file_counter += 1
    
#---------------parse to 10000 sequences per file---------------------   

In [ ]:
#---------------------------Dataset Class-----------------------------------

class singlefile(data.Dataset):
    
    
    def __init__(self, data_root, filename):
        
        self.path = os.path.join(data_root, filename)
        self.xy = []
        
        f = open(self.path, 'r')
        for line in f:
            if 'positive' in data_root:
                temp_list = self.one_hot(line.strip('\n'))
                self.xy.append([np.transpose(torch.Tensor(temp_list)), torch.Tensor([0,1])])
            if 'negative' in data_root:
                temp_list = self.one_hot(line.strip('\n'))
                self.xy.append([np.transpose(torch.Tensor(temp_list)), torch.Tensor([1,0])])

        
        
    def one_hot(self, seq):
        encoding_map = {'A': [1,0,0,0,0], 'T': [0,1,0,0,0], 'C': [0,0,1,0,0], 'G': [0,0,0,1,0], 'N': [0,0,0,0,1]}
        temp_list = []
        for s in seq:
            temp_list.append(encoding_map[s])
        return temp_list
               
    def __getitem__(self, index):
        return self.xy[index][0], self.xy[index][1]
        #return self.xy[index]
        
    def __len__(self):
        return len(self.xy)
    

        
    
#test = singlefile('processed/positive', 'positive_500.fa')
#print(test.__getitem__(3))


listofdatasets = []
for i, j in zip(os.listdir('processed/positive'), os.listdir('processed/negative')):
    #listofdatasets = []
    if not i.endswith('.fa'):
        continue
    if not j.endswith('.fa'):
        continue
        
    listofdatasets.append(singlefile('processed/positive', i))
    listofdatasets.append(singlefile('processed/negative', j))
    

    
    
    #break
    
random.shuffle(listofdatasets)
concat_dataset = data.ConcatDataset(listofdatasets)

print('saving')
with open('concat_dataset.pickle', 'wb') as handle:
    pickle.dump(concat_dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)
    


print(len(concat_dataset))
#print(concat_dataset[1999].__getitem__(1))
#print(concat_dataset[1999].__getitem__(0))


#---------------------------Dataset Class-----------------------------------   

In [50]:
class allfiles(data.Dataset):
    
    
    def __init__(self, pos_data_dir, neg_data_dir):
        
        self.list_index_counter = 0
        self.combined_list = []
        self.pos_filename = os.listdir(pos_data_dir)
        self.neg_filename = os.listdir(neg_data_dir)
        self.num_files = len(self.pos_filename)
        self.pos_filepath = os.path.join(pos_data_dir, self.pos_filename[self.list_index_counter]) 
        self.neg_filepath = os.path.join(neg_data_dir, self.neg_filename[self.list_index_counter])
        
        f = open(self.pos_filepath, 'r')
        for line in f:
            temp_seq = self.one_hot(line.strip('\n'))
            self.combined_list.append([np.transpose(torch.Tensor(temp_seq)), torch.Tensor([0,1])])
            
        f = open(self.neg_filepath, 'r') 
        for line in f:
            temp_seq = self.one_hot(line.strip('\n'))
            self.combined_list.append([np.transpose(torch.Tensor(temp_seq)), torch.Tensor([1,0])])
        
        self.list_index_counter += 1    
        random.shuffle(self.combined_list)        
        
    def one_hot(self, seq):
        encoding_map = {'A': [1,0,0,0,0], 'T': [0,1,0,0,0], 'C': [0,0,1,0,0], 'G': [0,0,0,1,0], 'N': [0,0,0,0,1]}
        temp_list = []
        for s in seq:
            temp_list.append(encoding_map[s])
        return temp_list
               
    def __getitem__(self, index):
        if self.list_index_counter < self.num_files:
            if len(self.combined_list) < 15000:
                self.pos_filepath = os.path.join(pos_data_dir, self.pos_filename[self.list_index_counter]) 
                self.neg_filepath = os.path.join(neg_data_dir, self.neg_filename[self.list_index_counter])
                f = open(self.pos_filepath, 'r')
                for line in f:
                    temp_seq = self.one_hot(line.strip('\n'))
                    self.combined_list.append([np.transpose(torch.Tensor(temp_seq)), torch.Tensor([0,1])])

                f = open(self.neg_filepath, 'r') 
                for line in f:
                    temp_seq = self.one_hot(line.strip('\n'))
                    self.combined_list.append([np.transpose(torch.Tensor(temp_seq)), torch.Tensor([1,0])])
                self.list_index_counter += 1
            
            
        random.shuffle(self.combined_list) 
        selected_index = self.combined_list[index]
        self.combined_list.pop(index)
         
        return selected_index

    def __len__(self):
        return len(self.combined_list)
    
test = allfiles('processed/positive', 'processed/negative')
print(len(test.combined_list))
test.__getitem__(5)
print(len(test.combined_list))



20000
19999


In [3]:
#-----------------------Model Class----------------------------

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv1d(5, 16, 11, stride=1, padding=5)
        self.conv2 = nn.Conv1d(16, 16, 11, stride=1, padding=5)
        self.drop1 = nn.Dropout(0.25)
        self.conv3 = nn.Conv1d(16, 16, 11, stride=1, padding=5)
        self.conv4 = nn.Conv1d(16, 16, 11, stride=1, padding=5)
        self.drop2 = nn.Dropout(0.25)
        self.conv5 = nn.Conv1d(16, 16, 11, stride=1, padding=5)
        self.conv6 = nn.Conv1d(16, 16, 11, stride=1, padding=5)
        self.drop3 = nn.Dropout(0.25)
        
    
        x = torch.randn((5, 101)).view(-1, 5 , 101)
        self._to_linear = None
        self.convs(x)
        self.fc1 = nn.Linear(self._to_linear, 500)
        self.fc2 = nn.Linear(500,2)
        
    def convs(self, x):
        
        x = F.max_pool1d(F.relu(self.conv2(F.relu(self.conv1(x)))), 2)
        x = (self.drop1(x))
        #print(x.shape)
        
        x = F.max_pool1d(F.relu(self.conv4(F.relu(self.conv3(x)))),  2)
        x = (self.drop2(x))
        #print(x.shape)
        
        x = F.max_pool1d(F.relu(self.conv6(F.relu(self.conv5(x)))),  2)
        x = (self.drop3(x))
        #print(x.shape)
        
        if self._to_linear is None:
           # print(x[0].shape)
            self._to_linear = x[0].shape[0]*x[0].shape[1]
            #print(self._to_linear)
        return x
        
                    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
#-----------------------Model Class----------------------------     

In [4]:
#------initialize model and hyperparameters--------

net = Net()
print(net)


optimizer = optim.Adam(net.parameters(), lr=0.001)
#loss_function = nn.MSELoss()
loss_function = nn.BCELoss()

batch_size = 64
EPOCHS = 30

#------initialize model and hyperparameters--------

Net(
  (conv1): Conv1d(5, 16, kernel_size=(11,), stride=(1,), padding=(5,))
  (conv2): Conv1d(16, 16, kernel_size=(11,), stride=(1,), padding=(5,))
  (drop1): Dropout(p=0.25, inplace=False)
  (conv3): Conv1d(16, 16, kernel_size=(11,), stride=(1,), padding=(5,))
  (conv4): Conv1d(16, 16, kernel_size=(11,), stride=(1,), padding=(5,))
  (drop2): Dropout(p=0.25, inplace=False)
  (conv5): Conv1d(16, 16, kernel_size=(11,), stride=(1,), padding=(5,))
  (conv6): Conv1d(16, 16, kernel_size=(11,), stride=(1,), padding=(5,))
  (drop3): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=192, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=2, bias=True)
)


In [ ]:
#--------initialize data for single fasta--------------

x = torch.Tensor([i[0] for i in data1]).view(-1, 5, 101)
y = torch.Tensor([i[1] for i in data1])

#--------initialize data for single fasta--------------

In [ ]:
#----------------training for single fasta------------------

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(x), batch_size)):
        batchx = x[i : i+batch_size].view(-1, 5, 101)
        batchy = y[i : i+batch_size]


        net.zero_grad()
        
        outputs = net.forward(batchx)
        loss = loss_function(outputs, batchy)
        loss.backward()
        optimizer.step()
        
    print(f"Epoch: {epoch}. Loss: {loss}")
    
#----------------training for single fasta------------------

In [5]:
#----------------------------Model Training--------------------------------

seqdataloader = data.DataLoader(concat_dataset, batch_size, shuffle = True)



for epoch in range(1, EPOCHS + 1):
    
    correct = 0
    all_label = []
    all_pred = []
    
    for i, (inputs, label) in enumerate(seqdataloader):
        
        
        inputs = inputs.view(-1, 5, 101)
        
        #print(inputs)
        #inputs = torch.Tensor(inputs[0])
        #print(len(inputs[0][0]))
        #inputs = inputs.view(-1, 5, 101)
        
        #print(label)
        net.zero_grad()
        outputs = net.forward(inputs)
        #print(outputs)
        loss = loss_function(outputs, label)
        loss.backward()
        optimizer.step()
        
        #---------------------AUC------------------------
        pred = outputs.argmax(dim=1, keepdim=True)
        real_labels = label.argmax(dim=1, keepdim= True)
        
        #for i in range(len(real_labels)):
            #print(real_labels[i], pred[i])
            
        correct += pred.eq(real_labels).sum().item()
        #print(correct)
        all_label.extend(real_labels.tolist())
        all_pred.extend(pred.tolist())
        #print(len(all_label))
        #------------------------------------------------
    #print(correct)
    #print(len(all_label))
    #print(correct/len(all_label))
    print(f"Epoch: {epoch}. Loss: {loss}")
    print("Train AUC score: {:.4f}".format(roc_auc_score(np.array(all_label), np.array(all_pred))))
    acc = correct/len(all_label)
    print('Accuracy: ' ,correct/len(all_label))
         
#----------------------------Model Training--------------------------------    

Epoch: 1. Loss: 0.6560514569282532
Train AUC score: 0.7095
Accuracy:  0.70945
Epoch: 2. Loss: 0.45818066596984863
Train AUC score: 0.7767
Accuracy:  0.7767
Epoch: 3. Loss: 0.30576083064079285
Train AUC score: 0.7950
Accuracy:  0.795
Epoch: 4. Loss: 0.3440389037132263
Train AUC score: 0.8059
Accuracy:  0.80595
Epoch: 5. Loss: 0.5110734701156616
Train AUC score: 0.8127
Accuracy:  0.8127
Epoch: 6. Loss: 0.2087555229663849
Train AUC score: 0.8152
Accuracy:  0.8152
Epoch: 7. Loss: 0.4304315745830536
Train AUC score: 0.8188
Accuracy:  0.8188
Epoch: 8. Loss: 0.24490857124328613
Train AUC score: 0.8227
Accuracy:  0.8227
Epoch: 9. Loss: 0.5164089202880859
Train AUC score: 0.8283
Accuracy:  0.82835


KeyboardInterrupt: 